In [1]:
import pandas
import pickle

In [2]:
with open('../raw_data/test_df.pickle', 'rb') as f:
            test_df = pickle.load(f)

In [ ]:
    def from_coord_to_matrix_train(self, lat_meters, lon_meters):
        """
        outputs the 3D matrix of all coordinates for a given bucket height and width in meters
        """
        self.lat_meters = lat_meters
        self.lon_meters = lon_meters

        df = self.train_df.copy()
        
        # add 'time_index' column to df
        ind = {time: index for index, time in enumerate(np.sort(df['six_hour_date'].unique()))}
        df['time_index'] = df['six_hour_date'].map(ind)

        #initiate matrix
        grid_offset = np.array([ -df['latitude'].max() , df['longitude'].min(), 0 ]) # Where do you start
        #from meters to lat/lon step

        # lat_spacing, lon_spacing = self.from_meters_to_steps()
        # Added from_meters_to_steps in utils instead: CLEAN UP FOR TEST AS WELL IF IT WORKS
        lat_spacing, lon_spacing = from_meters_to_steps(lat_meters, lon_meters)

        grid_spacing = np.array([lat_spacing , lon_spacing, 1 ]) # What's the space you consider (euclidian here)
        #get points coordinates
        coords = np.array([( -lat, lon,t_ind) for lat, lon,t_ind \
                       in zip(df['latitude'],df['longitude'],df['time_index'])])

        # Convert point to index
        indexes = np.round((coords - grid_offset)/grid_spacing).astype('int')
        X = indexes[:,0]
        Y = indexes[:,1]
        Z = indexes[:,2]

        #virgin matrix

        # 75th precinct distances:
        lat_min, lat_max, lon_max, lon_min = (40.6218192717505,
                                              40.6951504231971,
                                              -73.90404639808888,
                                              -73.83559344190869)

        lat_diff = lat_max - lat_min # distance in lat that makes up width of precinct 75
        lon_diff = lon_min - lon_max # distance in lon that makes up width of precinct 75

        # dim 1: distance of precinct in lat / lat_spacing
        a = np.zeros((np.round(lat_diff / lat_spacing).astype('int') + 1,
                     np.round(lon_diff / lon_spacing).astype('int') + 1,
                     Z.max() + 1))

        # old version: a = np.zeros((X.max()+1, Y.max()+1, Z.max()+1))

        a[X, Y, Z] = 1

        self.lat_size = a.shape[1]
        self.lon_size = a.shape[2]
        self.img3D_non_conv_train = a

        return self.img3D_non_conv_train

In [ ]:



def gaussian_filtering_train(self):
    '''
      Returns img3D convoluted
    '''

    self.img3D_conv_train = gaussian_filter(self.img3D_non_conv_train,
        sigma = (self.sigma_x, self.sigma_y, self.sigma_z))

    return self.img3D_conv_train

########
# Input: Stacking et all.

def stacking_train(self, window, lat_step, lon_step, time_step):
    '''
        Returns stacked crimes.
    '''
    grid_offset = np.array([0,0,0]) # Where do you start
    #new steps from precise grid
    grid_spacing = np.array([lat_step , lon_step, time_step])
    #get points coordinates
    coords = np.argwhere(window)
    flat = window.flatten()
    values = flat[flat !=0]
    # Convert point to index
    indexes = np.round((coords - grid_offset)/grid_spacing).astype('int')
    X = indexes[:,0]
    Y = indexes[:,1]
    Z = indexes[:,2]
    #virgin matrix: 256 is arbitrary size that works in model
    stacked_crimes = np.zeros((192, 132, Z.max() + 2))

    for i in range(len(indexes)):

        if stacked_crimes[X[i], Y[i], Z[i]] == 0:
            stacked_crimes[X[i], Y[i], Z[i]] = values[i]
        else:
            stacked_crimes[X[i], Y[i], Z[i]] += values[i]

    return stacked_crimes

def get_observation_target_train(self,
                       obs_timeframe,obs_lat,obs_lon, obs_time,
                       target_timeframe,  tar_lat,tar_lon, tar_time):
    '''
    output an observation of x_length consecutive images and the y_length next images as the target
    obs_step, obs_timeframe, target_step, target_timeframe : unit = hours
    '''

    # sample length to absorb impact of gaussian time sigma
    sample_length = obs_timeframe + (self.raw_z + 1) + target_timeframe

    # finds starting position
    position = np.random.randint(0, self.img3D_conv_train.shape[2] - sample_length)

    # samples in train and test dfs
    subsample = self.img3D_conv_train[:, :, position : position + sample_length]

    # divide the subsample in X and y
    observations = subsample[:, :, : obs_timeframe]

    targets = subsample[:, :, - target_timeframe : ]

    # stacked images
    observation = self.stacking_train(observations, obs_lat, obs_lon, obs_time)

    target = self.stacking_train(targets, tar_lat, tar_lon, tar_time)

    return observation, target

def get_X_y_train(self, nb_observations_train, obs_tf,obs_lat,obs_lon, obs_time,
                tar_tf, tar_lat,tar_lon, tar_time):
    '''
    outputs n observations and their associated targets
    '''
    X = []
    y = []

    for n in range(nb_observations_train):
        print(f'Creating observation {n} out of {nb_observations_train}')
        X_subsample, y_subsample = self.get_observation_target_train(obs_tf,
                                    obs_lat,obs_lon, obs_time,
                                    tar_tf,  tar_lat,tar_lon, tar_time)
        X.append(X_subsample)
        y.append(y_subsample)

    X = np.array(X)
    y = np.array(y)

    self.X_train = X
    self.y_train = y

    return self.X_train, self.y_train